In [1]:
import numpy as np
import pandas as pd
import h5py
import scipy
import scanpy as sc
import anndata
import pkg_resources
from scbasset.utils import *

# plotting functions
import seaborn as sns
import matplotlib.pyplot as plt

import os

### read example from 10x multiome output

In [2]:
data_path = 'data/multiome_example/'

h5_file = data_path + '/raw_multiome/pbmc_granulocyte_sorted_3k_filtered_feature_bc_matrix.h5'
bed_file = data_path + '/raw_multiome/pbmc_granulocyte_sorted_3k_atac_peaks.bed'

In [3]:
peak = pd.read_csv(bed_file, sep='\t', names=['chr','start','end'])
ad = sc.read_10x_h5(h5_file, gex_only=False)
ad.write_csvs(data_path)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
writing .csv files to data/multiome_example


In [4]:
peak

,chr,start,end
0,chr1,9986,10403
1,chr1,180690,181695
2,chr1,191335,192015
3,chr1,267775,268224
4,chr1,271144,271510
...,...,...,...
156602,KI270713.1,20252,22762
156603,KI270713.1,26456,28921
156604,KI270713.1,29478,30798
156605,KI270713.1,31532,33156


### filtering

In [5]:
ad_rna = ad[:, ad.var['feature_types']=='Gene Expression']
ad_atac = ad[:, ad.var['feature_types']=='Peaks']
ad_atac.var['chr'] = peak['chr'].values
ad_atac.var['start'] = peak['start'].values
ad_atac.var['end'] = peak['end'].values
ad_rna

Trying to set attribute `.var` of view, copying.


View of AnnData object with n_obs × n_vars = 2714 × 36601
    var: 'gene_ids', 'feature_types', 'genome'

In [6]:
ad_atac

AnnData object with n_obs × n_vars = 2714 × 156607
    var: 'gene_ids', 'feature_types', 'genome', 'chr', 'start', 'end'

In [7]:
# basic stats
sc.pp.filter_cells(ad_rna, min_genes=0)
sc.pp.filter_genes(ad_rna, min_cells=0)
sc.pp.filter_cells(ad_atac, min_genes=0)
sc.pp.filter_genes(ad_atac, min_cells=0)
ad_rna.var.loc[:,:]

Trying to set attribute `.obs` of view, copying.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


,gene_ids,feature_types,genome,n_cells
MIR1302-2HG,ENSG00000243485,Gene Expression,GRCh38,0
FAM138A,ENSG00000237613,Gene Expression,GRCh38,0
OR4F5,ENSG00000186092,Gene Expression,GRCh38,0
AL627309.1,ENSG00000238009,Gene Expression,GRCh38,28
AL627309.3,ENSG00000239945,Gene Expression,GRCh38,0
...,...,...,...,...
AC141272.1,ENSG00000277836,Gene Expression,GRCh38,1
AC023491.2,ENSG00000278633,Gene Expression,GRCh38,0
AC007325.1,ENSG00000276017,Gene Expression,GRCh38,1
AC007325.4,ENSG00000278817,Gene Expression,GRCh38,23


In [8]:
ad_atac

AnnData object with n_obs × n_vars = 2714 × 156607
    obs: 'n_genes'
    var: 'gene_ids', 'feature_types', 'genome', 'chr', 'start', 'end', 'n_cells'

In [9]:
# a gene need to be expressed in 5% cells
# a peak need to be accessible in 5% cells
thres = int(ad.shape[0]*0.05)
ad_rna = ad_rna[:, ad_rna.var['n_cells']>thres]
ad_atac = ad_atac[:, ad_atac.var['n_cells']>thres]
ad_rna

View of AnnData object with n_obs × n_vars = 2714 × 8652
    obs: 'n_genes'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'

In [10]:
ad_atac

View of AnnData object with n_obs × n_vars = 2714 × 27157
    obs: 'n_genes'
    var: 'gene_ids', 'feature_types', 'genome', 'chr', 'start', 'end', 'n_cells'

In [11]:
ad_atac.obs.loc[:,:]

,n_genes
AAACAGCCAAATATCC-1,5241
AAACAGCCAGGAACTG-1,9587
AAACAGCCAGGCTTCG-1,9956
AAACCAACACCTGCTC-1,6364
AAACCAACAGATTCAT-1,6941
...,...
TTTGTGGCATTAGCCA-1,6514
TTTGTGGCATTGCGAC-1,4696
TTTGTGTTCCGCCTAT-1,7200
TTTGTGTTCCGTGACA-1,6688


In [12]:
ad_atac.var.loc[:,:]

,gene_ids,feature_types,genome,chr,start,end,n_cells
chr1:633785-634272,chr1:633785-634272,Peaks,GRCh38,chr1,633785,634272,211
chr1:777438-779964,chr1:777438-779964,Peaks,GRCh38,chr1,777438,779964,1111
chr1:816129-817671,chr1:816129-817671,Peaks,GRCh38,chr1,816129,817671,167
chr1:819740-823653,chr1:819740-823653,Peaks,GRCh38,chr1,819740,823653,157
chr1:826607-828042,chr1:826607-828042,Peaks,GRCh38,chr1,826607,828042,568
...,...,...,...,...,...,...,...
GL000195.1:30120-33432,GL000195.1:30120-33432,Peaks,GRCh38,GL000195.1,30120,33432,706
GL000219.1:40022-43206,GL000219.1:40022-43206,Peaks,GRCh38,GL000219.1,40022,43206,758
GL000219.1:44398-46512,GL000219.1:44398-46512,Peaks,GRCh38,GL000219.1,44398,46512,165
GL000219.1:98576-101114,GL000219.1:98576-101114,Peaks,GRCh38,GL000219.1,98576,101114,1356


In [29]:
pd.DataFrame(ad_atac.X,)

,0
0,"(0, 4)\t2.0\n (0, 15)\t4.0\n (0, 20)\t4.0\..."
1,"(0, 2)\t2.0\n (0, 10)\t4.0\n (0, 14)\t2.0\..."
2,"(0, 1)\t2.0\n (0, 9)\t2.0\n (0, 13)\t2.0\n..."
3,"(0, 1)\t2.0\n (0, 7)\t2.0\n (0, 10)\t2.0\n..."
4,"(0, 1)\t4.0\n (0, 8)\t2.0\n (0, 14)\t2.0\n..."
...,...
2709,"(0, 1)\t2.0\n (0, 5)\t2.0\n (0, 7)\t2.0\n ..."
2710,"(0, 1)\t2.0\n (0, 13)\t2.0\n (0, 15)\t2.0\..."
2711,"(0, 1)\t4.0\n (0, 3)\t1.0\n (0, 7)\t2.0\n ..."
2712,"(0, 1)\t2.0\n (0, 4)\t2.0\n (0, 7)\t2.0\n ..."


### save h5ad and filtered bed

In [14]:
os.makedirs('%s/processed'%data_path, exist_ok=True)

chrs = ['chr'+str(i) for i in range(1,23)] + ['chrX', 'chrY']
chrs

['chr1',
 'chr2',
 'chr3',
 'chr4',
 'chr5',
 'chr6',
 'chr7',
 'chr8',
 'chr9',
 'chr10',
 'chr11',
 'chr12',
 'chr13',
 'chr14',
 'chr15',
 'chr16',
 'chr17',
 'chr18',
 'chr19',
 'chr20',
 'chr21',
 'chr22',
 'chrX',
 'chrY']

In [15]:
ad_atac = ad_atac[:, ad_atac.var['chr'].isin(chrs)]
ad_atac

View of AnnData object with n_obs × n_vars = 2714 × 27150
    obs: 'n_genes'
    var: 'gene_ids', 'feature_types', 'genome', 'chr', 'start', 'end', 'n_cells'

In [16]:
ad_atac.write('%s/processed/pbmc_multiome_ad.h5ad'%data_path)
ad_atac.var.loc[:,['chr','start','end']].to_csv('%s/processed/pbmc_multiome_peaks.bed'%data_path, header=False, sep='\t', index=False)

/staging/leuven/stg_00002/lcb/hsnyers/software/anaconda3/envs/torch_scBasset/lib/python3.8/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
/staging/leuven/stg_00002/lcb/hsnyers/software/anaconda3/envs/torch_scBasset/lib/python3.8/site-packages/anndata/_core/anndata.py:1236: ImplicitModificationWarning: Initializing view as actual.
  warnings.warn(
Trying to set attribute `.var` of view, copying.
... storing 'feature_types' as categorical
/staging/leuven/stg_00002/lcb/hsnyers/software/anaconda3/envs/torch_scBasset/lib/python3.8/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will

### create inputs for scBasset

In [17]:
filtered_bed_file = '%s/processed/pbmc_multiome_peaks.bed'%data_path # path to save the filtered peak atlas
ad_file = '%s/processed/pbmc_multiome_ad.h5ad'%data_path
output_file ='%s/processed/pbmc_multiome_train_test_val.h5'%data_path

In [18]:
# fasta file depending the the genome build for dataset
# for hg38: please download hg38.fa.gz at: https://hgdownload.cse.ucsc.edu/goldenpath/hg38/bigZips/
# unzip and use fasta_file to indicate the path to hg38.fa
fasta_file = '/user/leuven/346/vsc34619/hsnyers/Github/torch_scBasset/data/multiome_example/hg38.fa/hg38.fa' 

In [19]:
make_h5(input_ad=ad_file, 
        input_bed=filtered_bed_file, 
        input_fasta=fasta_file, 
        out_file=output_file)

In [20]:
f = h5py.File(output_file, 'r')
X = f['X'][:].astype('float32')
Y = f['Y'][:].astype('float32')
f, X, Y


(<HDF5 file "pbmc_multiome_train_test_val.h5" (mode r)>,
 array([[[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [1., 0., 0., 0.],
         ...,
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.]],
 
        [[0., 0., 0., 1.],
         [0., 1., 0., 0.],
         [0., 0., 0., 1.],
         ...,
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 0., 1.]],
 
        [[0., 0., 0., 1.],
         [1., 0., 0., 0.],
         [0., 0., 0., 1.],
         ...,
         [0., 1., 0., 0.],
         [0., 0., 0., 1.],
         [0., 0., 1., 0.]],
 
        ...,
 
        [[0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         ...,
         [0., 0., 0., 1.],
         [1., 0., 0., 0.],
         [0., 0., 1., 0.]],
 
        [[0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         ...,
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 0., 1.]],
 
        [[1., 0., 0., 0.],
      

In [21]:
pd.DataFrame(X[0])

,0,1,2,3
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0
...,...,...,...,...
1339,0.0,0.0,0.0,1.0
1340,0.0,1.0,0.0,0.0
1341,0.0,1.0,0.0,0.0
1342,0.0,1.0,0.0,0.0


In [22]:
len(X)

27150

In [23]:
pd.DataFrame(Y)

,0,1,2,3,4,5,6,7,8,9,...,2704,2705,2706,2707,2708,2709,2710,2711,2712,2713
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27146,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
27147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
27148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:

# Split train-validation set    
train_ids = f['train_ids'][:]
val_ids = f['val_ids'][:]
test_ids = f['test_ids'][:]
train_ids, val_ids, test_ids

(array([    0,     1,     2, ..., 27147, 27148, 27149]),
 array([ 3174,   591,  7532, ..., 14850, 11720,  4365]),
 array([    7,    27,    61, ..., 27098, 27107, 27113]))

In [25]:

X_train = X[train_ids]
Y_train = Y[train_ids]
X_val = X[val_ids]
Y_val = Y[val_ids]
len(X_train)

24436

In [26]:

n_cells = Y.shape[1]
n_cells

2714

### alternatively you can download my pre-processed files from google cloud bucket

In [27]:
# import subprocess
# import os

# download_savepath = '../../data/download'
# os.makedirs(download_savepath, exist_ok=True)

# if not os.path.exists('%s/pbmc_multiome_ad.h5ad'%download_savepath):
#     subprocess.run('wget -P %s https://storage.googleapis.com/scbasset_tutorial_data/pbmc_multiome_ad.h5ad'%download_savepath, shell=True)

# if not os.path.exists('%s/pbmc_multiome_peaks.bed'%download_savepath):
#     subprocess.run('wget -P %s https://storage.googleapis.com/scbasset_tutorial_data/pbmc_multiome_peaks.bed'%download_savepath, shell=True)

# if not os.path.exists('%s/pbmc_multiome_train_test_val.h5'%download_savepath):
#     subprocess.run('wget -P %s https://storage.googleapis.com/scbasset_tutorial_data/pbmc_multiome_train_test_val.h5'%download_savepath, shell=True)
